In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold, ShuffleSplit
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from pytorch_tabnet.tab_model import TabNetRegressor

import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

import shap

import gc
gc.enable()

optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_str_lengths(10000)

polars.config.Config

# Preprocessing & Feature Engineering

In [2]:
#from IPython.display import display as ipydisplay, HTML
#ipydisplay(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 80em; }</style>"))

In [3]:
SEED = 42
n_splits = 5

In [4]:
path = "I:/Kaggle/child-mind-institute-problematic-internet-use/"

In [5]:
train_final_df = pd.read_csv(path + 'cat_train_selected_8.csv', dtype={'id': str})
test_final_df = pd.read_csv(path + 'cat_test_selected_8.csv', dtype={'id': str})

In [6]:
print(train_final_df.shape)
train_final_df.head()

(2736, 94)


,id,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_GSND,FGC-FGC_GSD,FGC-FGC_PU,BIA-BIA_Activity_Level_num,BIA-BIA_Fat,SDS-SDS_Total_Raw,PreInt_EduHx-computerinternet_hoursday,Physical-Waist_Circumference_isnan,PAQ_C-Season_isnan,enmo_mean,anglez_max,X_weekday_4_entropy,X_weekday_5_entropy,Z_weekday_1_min,Z_weekday_4_std,Z_weekday_5_std,enmo_weekday_1_15%,enmo_weekday_3_mean,light_weekday_1_std,light_weekday_3_max,light_weekday_4_max,light_weekday_4_entropy,battery_use_since_wear_weekday_3_mean,battery_use_since_wear_weekday_3_30%,battery_use_since_wear_weekday_5_max,X_time_of_day_hour_window_1_sum,X_time_of_day_hour_window_2_sum,X_time_of_day_hour_window_4_mean,X_time_of_day_hour_window_15_max,X_time_of_day_hour_window_19_entropy,Y_time_of_day_hour_window_0_sum,Y_time_of_day_hour_window_2_std,Y_time_of_day_hour_window_5_entropy,Y_time_of_day_hour_window_6_mean,Y_time_of_day_hour_window_8_sum,Y_time_of_day_hour_window_9_sum,Y_time_of_day_hour_window_9_entropy,Y_time_of_day_hour_window_11_mean,Y_time_of_day_hour_window_13_3%,Y_time_of_day_hour_window_15_min,Y_time_of_day_hour_window_20_sum,Y_time_of_day_hour_window_21_sum,Y_time_of_day_hour_window_21_entropy,Z_time_of_day_hour_window_0_min,Z_time_of_day_hour_window_1_mean,Z_time_of_day_hour_window_5_mean,Z_time_of_day_hour_window_5_sum,Z_time_of_day_hour_window_9_sum,Z_time_of_day_hour_window_11_std,Z_time_of_day_hour_window_15_std,Z_time_of_day_hour_window_16_max,Z_time_of_day_hour_window_21_mean,Z_time_of_day_hour_window_22_min,Z_time_of_day_hour_window_23_min,enmo_time_of_day_hour_window_4_std,enmo_time_of_day_hour_window_8_mean,enmo_time_of_day_hour_window_14_mean,enmo_time_of_day_hour_window_15_10%,enmo_time_of_day_hour_window_16_max,enmo_time_of_day_hour_window_18_mean,enmo_time_of_day_hour_window_18_max,anglez_time_of_day_hour_window_8_entropy,anglez_time_of_day_hour_window_12_entropy,light_time_of_day_hour_window_7_std,light_time_of_day_hour_window_7_70%,light_time_of_day_hour_window_8_mean,light_time_of_day_hour_window_8_std,light_time_of_day_hour_window_10_max,light_time_of_day_hour_window_12_1%,light_time_of_day_hour_window_13_std,light_time_of_day_hour_window_13_max,light_time_of_day_hour_window_19_35%,light_time_of_day_hour_window_19_max,light_time_of_day_hour_window_20_std,light_time_of_day_hour_window_21_std,battery_use_since_wear_time_of_day_hour_window_8_mean,battery_use_since_wear_time_of_day_hour_window_14_1%,battery_use_since_wear_time_of_day_hour_window_16_min,battery_use_since_wear_time_of_day_hour_window_18_5%,PAQ_C-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,FMI_BFP,Muscle_to_Fat,sii
0,00008ff9,5.0,51.0,16.877316,46.0,50.8,25.0,130.2,0.0,20.50,21.18,0.0,2.0,9.213770,41.6,3.0,1.0,1.0,0.045088,89.296684,39.926039,34.137011,-1.001339,0.545945,0.502684,0.002152,0.046002,30.577977,1519.440030,929.050000,9.275002,213.972368,136.85000,696.71665,-211.553589,239.102159,-0.011758,1.140360,22.976666,-43.818953,0.304632,23.115072,-0.007757,-332.684457,-156.169044,52.527369,0.077756,-0.815943,-1.295111,-18.909606,228.457314,16.746607,-0.989829,-0.219695,-0.026439,232.212733,-752.212750,0.494078,0.513271,1.461590,0.019698,-0.601909,-0.973880,0.021542,0.044125,0.061529,0.001617,2.518643,0.052438,1.742300,55.225293,17.738485,30.704474,38.370415,15.214690,25.508007,1343.000024,0.073666,39.113261,1403.999985,2.858217,86.159849,13.298962,24.520308,286.112347,6.200000,9.6,33.334136,0.0,1.0,0.0,0.332267,6.383063,2.0
1,000fd460,9.0,63.2,14.035590,48.0,46.0,22.0,122.0,3.0,23.08,24.94,5.0,2.0,3.970850,46.0,0.0,0.0,0.0,0.052679,89.456932,18.003700,18.304327,-1.000690,0.529854,0.540959,0.003071,0.043606,62.688214,1090.320020,1148.777557,8.891857,316.855106,284.86665,507.16665,-98.813717,383.965568,0.007261,1.058457,17.475854,-306.613574,0.404812,25.279001,0.055939,151.477521,184.093170,59.785731,0.028424,-0.885720,-1.615314,-51.054434,223.565190,17.279529,-0.984249,-0.052453,0.

In [7]:
print(test_final_df.shape)
test_final_df.head()

(20, 93)


,id,Basic_Demos-Age,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Systolic_BP,FGC-FGC_CU,FGC-FGC_GSND,FGC-FGC_GSD,FGC-FGC_PU,BIA-BIA_Activity_Level_num,BIA-BIA_Fat,SDS-SDS_Total_Raw,PreInt_EduHx-computerinternet_hoursday,Physical-Waist_Circumference_isnan,PAQ_C-Season_isnan,enmo_mean,anglez_max,X_weekday_4_entropy,X_weekday_5_entropy,Z_weekday_1_min,Z_weekday_4_std,Z_weekday_5_std,enmo_weekday_1_15%,enmo_weekday_3_mean,light_weekday_1_std,light_weekday_3_max,light_weekday_4_max,light_weekday_4_entropy,battery_use_since_wear_weekday_3_mean,battery_use_since_wear_weekday_3_30%,battery_use_since_wear_weekday_5_max,X_time_of_day_hour_window_1_sum,X_time_of_day_hour_window_2_sum,X_time_of_day_hour_window_4_mean,X_time_of_day_hour_window_15_max,X_time_of_day_hour_window_19_entropy,Y_time_of_day_hour_window_0_sum,Y_time_of_day_hour_window_2_std,Y_time_of_day_hour_window_5_entropy,Y_time_of_day_hour_window_6_mean,Y_time_of_day_hour_window_8_sum,Y_time_of_day_hour_window_9_sum,Y_time_of_day_hour_window_9_entropy,Y_time_of_day_hour_window_11_mean,Y_time_of_day_hour_window_13_3%,Y_time_of_day_hour_window_15_min,Y_time_of_day_hour_window_20_sum,Y_time_of_day_hour_window_21_sum,Y_time_of_day_hour_window_21_entropy,Z_time_of_day_hour_window_0_min,Z_time_of_day_hour_window_1_mean,Z_time_of_day_hour_window_5_mean,Z_time_of_day_hour_window_5_sum,Z_time_of_day_hour_window_9_sum,Z_time_of_day_hour_window_11_std,Z_time_of_day_hour_window_15_std,Z_time_of_day_hour_window_16_max,Z_time_of_day_hour_window_21_mean,Z_time_of_day_hour_window_22_min,Z_time_of_day_hour_window_23_min,enmo_time_of_day_hour_window_4_std,enmo_time_of_day_hour_window_8_mean,enmo_time_of_day_hour_window_14_mean,enmo_time_of_day_hour_window_15_10%,enmo_time_of_day_hour_window_16_max,enmo_time_of_day_hour_window_18_mean,enmo_time_of_day_hour_window_18_max,anglez_time_of_day_hour_window_8_entropy,anglez_time_of_day_hour_window_12_entropy,light_time_of_day_hour_window_7_std,light_time_of_day_hour_window_7_70%,light_time_of_day_hour_window_8_mean,light_time_of_day_hour_window_8_std,light_time_of_day_hour_window_10_max,light_time_of_day_hour_window_12_1%,light_time_of_day_hour_window_13_std,light_time_of_day_hour_window_13_max,light_time_of_day_hour_window_19_35%,light_time_of_day_hour_window_19_max,light_time_of_day_hour_window_20_std,light_time_of_day_hour_window_21_std,battery_use_since_wear_time_of_day_hour_window_8_mean,battery_use_since_wear_time_of_day_hour_window_14_1%,battery_use_since_wear_time_of_day_hour_window_16_min,battery_use_since_wear_time_of_day_hour_window_18_5%,PAQ_C-Season_Winter,Basic_Demos-Sex_0,Basic_Demos-Sex_1,FMI_BFP,Muscle_to_Fat
0,00008ff9,5.0,51.0,16.877316,46.000,50.8,25.0,130.2,0.0,20.50,21.18,0.0,2.0,9.213770,41.6,3.0,1.0,1.0,0.045088,89.296684,39.926039,34.137011,-1.001339,0.545945,0.502684,0.002152,0.046002,30.577977,1519.440030,929.050000,9.275002,213.972368,136.85000,696.71665,-211.553589,239.102159,-0.011758,1.140360,22.976666,-43.818953,0.304632,23.115072,-0.007757,-332.684457,-156.169044,52.527369,0.077756,-0.815943,-1.295111,-18.909606,228.457314,16.746607,-0.989829,-0.219695,-0.026439,232.212733,-752.212750,0.494078,0.513271,1.461590,0.019698,-0.601909,-0.973880,0.021542,0.044125,0.061529,0.001617,2.518643,0.052438,1.742300,55.225293,17.738485,30.704474,38.370415,15.214690,25.508007,1343.000024,0.073666,39.113261,1403.999985,2.858217,86.159849,13.298962,24.520308,286.112347,6.200000,9.6,33.334136,0.0,1.0,0.0,0.332267,6.383063
1,000fd460,9.0,63.2,14.035590,48.000,46.0,22.0,122.0,3.0,23.08,24.94,5.0,2.0,3.970850,46.0,0.0,0.0,0.0,0.052679,89.456932,18.003700,18.304327,-1.000690,0.529854,0.540959,0.003071,0.043606,62.688214,1090.320020,1148.777557,8.891857,316.855106,284.86665,507.16665,-98.813717,383.965568,0.007261,1.058457,17.475854,-306.613574,0.404812,25.279001,0.055939,151.477521,184.093170,59.785731,0.028424,-0.885720,-1.615314,-51.054434,223.565190,17.279529,-0.984249,-0.052453,0.0905

In [ ]:
train = pd.read_csv(path + 'train.csv', dtype={'id': str})
test = pd.read_csv(path + 'test.csv', dtype={'id': str})
#sample = pd.read_csv(path + 'sample_submission.csv', dtype={'id': str})

In [ ]:
data_dict = pd.read_csv(path + 'data_dictionary.csv')

In [ ]:
train_target = train[test.columns]

In [ ]:
concat_df = pd.concat([train_target, test], ignore_index=True)

In [ ]:
concat_df['total_num_nan'] = concat_df.isna().sum(axis=1)

In [ ]:
del train_target
gc.collect()

In [ ]:
nan_df = concat_df[concat_df['total_num_nan'] == concat_df['total_num_nan'].max()]

In [ ]:
for col in concat_df.columns:
    if col == 'id':
        continue
    new_col_name = col + '_isnan'
    concat_df[new_col_name] = concat_df[col].isna().astype(int)

In [ ]:
instruments = data_dict['Instrument'].unique().tolist()
instruments.remove('Identifier')

In [ ]:
for instrument in instruments:
    if 'Parent-Child Internet Addiction Test' == instrument:
        continue
    instrument_df = data_dict[data_dict['Instrument'] == instrument]
    field_list = instrument_df['Field'].unique().tolist()
    concat_df[instrument + '_num_nan'] = concat_df[field_list].isna().sum(axis=1)

del instrument_df, field_list
gc.collect()

In [ ]:
train_parquet_id_folders_list = os.listdir(path + 'series_train.parquet/')

In [ ]:
train_parquet_id_list = [v.replace('id=', '') for v in train_parquet_id_folders_list]

In [ ]:
test_parquet_id_folders_list = os.listdir(path + 'series_test.parquet/')

In [ ]:
test_parquet_id_list = [v.replace('id=', '') for v in test_parquet_id_folders_list]

In [ ]:
train_temp_df = concat_df.head(train.shape[0]).reset_index(drop=True)

In [ ]:
test_temp_df = concat_df.tail(test.shape[0]).reset_index(drop=True)

In [ ]:
del concat_df
gc.collect()

In [ ]:
train_temp_df['has_parquet'] = train_temp_df['id'].isin(train_parquet_id_list).astype(int)

In [ ]:
test_temp_df['has_parquet'] = test_temp_df['id'].isin(test_parquet_id_list).astype(int)

In [ ]:
# https://www.kaggle.com/code/antoninadolgorukova/cmi-piu-actigraphy-data-eda
entropy = lambda x: -(x / x.sum() * np.log(x / x.sum() + 1e-9)).sum()

In [ ]:
def feat_engi(df, target_cols):
    stats_df = df[target_cols].describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99]).T
    count = stats_df['count'][0]
    stats_df.drop('count', axis=1, inplace=True)
    
    column_names_list = []
    for idx in stats_df.index:
        for col in stats_df.columns:
            column_names_list.append(str(idx) + '_' + str(col))

    reshaped_df = pd.DataFrame(stats_df.values.reshape(-1)).T.reset_index(drop=True)
    reshaped_df.columns = column_names_list

    reshaped_df['count'] = count

    return reshaped_df

In [ ]:
def groupby_feat_engi(df, col_to_group, target_col):
    stats_df = pd.DataFrame(
        df.groupby(col_to_group)[target_col].describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99])
    )

    stats_df['sum'] = df.groupby(col_to_group)[target_col].sum()
    stats_df['entropy'] = df.groupby(col_to_group)[target_col].apply(entropy)
    
    column_names_list = []
    for idx in stats_df.index:
        for col in stats_df.columns:
            column_names_list.append(target_col + '_' + col_to_group + '_' + str(idx) + '_' + str(col))

    reshaped_df = pd.DataFrame(stats_df.values.reshape(-1)).T.reset_index(drop=True)
    reshaped_df.columns = column_names_list

    return reshaped_df

In [ ]:
def uncommon_groupby_feat_engi(df, col_to_group, target_col):
    stats_df = pd.DataFrame(
        df.groupby(col_to_group)[target_col].describe()
        #df.groupby(col_to_group)[target_col].describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99])
    )

    stats_df['sum'] = df.groupby(col_to_group)[target_col].sum()
    stats_df['entropy'] = df.groupby(col_to_group)[target_col].apply(entropy)

    
    #stats_stats_df = stats_df.describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99])
    stats_stats_df = stats_df.describe()
    stats_stats_count = stats_stats_df['count'][0]
    stats_stats_df.drop('count', axis=0, inplace=True)
    stats_stats_df = stats_stats_df.T
    
    column_names_list = []
    for idx in stats_stats_df.index:
        for col in stats_stats_df.columns:
            column_names_list.append(target_col + '_' + col_to_group + '_' + str(idx) + '_' + str(col))
    
    reshaped_df = pd.DataFrame(stats_stats_df.values.reshape(-1)).T.reset_index(drop=True)
    reshaped_df.columns = column_names_list

    current_cols = reshaped_df.columns.tolist()
    count_col_name = target_col + '_' + col_to_group + '_count'
    reshaped_df[count_col_name] = stats_stats_count

    reshaped_df = reshaped_df[[count_col_name] + current_cols]

    return reshaped_df

In [ ]:
def read_parquet(dataset='train'):
    parquet_id_folders_list = os.listdir(path + 'series_' + dataset + '.parquet/')
    ts_list = []
    ts_features_list = []
    for id_folder in tqdm(parquet_id_folders_list):
        iid = id_folder.replace('id=', '')
    
        parquet_files_path = path + 'series_' + dataset + '.parquet/' + id_folder + '/'
        parquet_files_list = os.listdir(parquet_files_path)
    
        dfs_list = []
        for parquet_filename in parquet_files_list:
            if '.parquet' in parquet_filename:
                parquet_df = pd.read_parquet(parquet_files_path + parquet_filename)
                dfs_list.append(parquet_df)
        parquet_concat_df = pd.concat(dfs_list, ignore_index=True)
        current_cols = parquet_concat_df.columns.tolist()
        
        parquet_concat_df['id'] = iid
        
        parquet_concat_df = parquet_concat_df[['id'] + current_cols]
        
        parquet_concat_df['day_since_wear'] = (parquet_concat_df['relative_date_PCIAT'] - parquet_concat_df['relative_date_PCIAT'].min()).astype(int)
        parquet_concat_df['time_of_day_sec'] = parquet_concat_df['time_of_day'] / 1e9
        parquet_concat_df.drop('time_of_day', axis=1, inplace=True)
        parquet_concat_df['time_of_day_min'] = parquet_concat_df['time_of_day_sec'] / 60
        parquet_concat_df['time_of_day_hour'] = parquet_concat_df['time_of_day_min'] / 60
        parquet_concat_df['time_of_day_day'] = parquet_concat_df['time_of_day_hour'] / 24
        parquet_concat_df['time_of_day_hour_window'] = parquet_concat_df['time_of_day_hour'].astype(int)
        parquet_concat_df['time_of_day_min_window'] = parquet_concat_df['time_of_day_min'].astype(int)
        parquet_concat_df['time_of_day_15_min_window'] = (parquet_concat_df['time_of_day_min'] / 15).astype(int)
        parquet_concat_df['timestamp_day'] = parquet_concat_df['day_since_wear'] + parquet_concat_df['time_of_day_day']
    
        if parquet_concat_df['timestamp_day'].nunique() != parquet_concat_df.shape[0]:
            print('yes')
    
        parquet_concat_df['timestamp_hour'] = parquet_concat_df['timestamp_day'] * 24
        parquet_concat_df['timestamp_min'] = parquet_concat_df['timestamp_day'] * 24 * 60
        parquet_concat_df['timestamp_sec'] = parquet_concat_df['timestamp_day'] * 24 * 60 * 60
        parquet_concat_df['timestamp_15_min'] = parquet_concat_df['timestamp_day'] * 24 * (60/15)
    
        parquet_concat_df['timestamp_hour_window'] = parquet_concat_df['timestamp_hour'].astype(int)
        parquet_concat_df['timestamp_min_window'] = parquet_concat_df['timestamp_min'].astype(int)
        parquet_concat_df['timestamp_15_min_window'] = parquet_concat_df['timestamp_15_min'].astype(int)
    
        battery_voltage_start = parquet_concat_df['battery_voltage'][0]
        parquet_concat_df['battery_use_since_wear'] = -(parquet_concat_df['battery_voltage'] - battery_voltage_start)
        
    
        worn_df = parquet_concat_df[parquet_concat_df['non-wear_flag'] == 0].reset_index(drop=True)
    
        
        feature_cols = ['X', 'Y', 'Z', 'enmo', 'anglez', 'light', 'battery_use_since_wear']
    
        features_df = feat_engi(worn_df, feature_cols)
    
        groupby_cols = ['weekday', 'time_of_day_hour_window']#, 'time_of_day_15_min_window']
    
        groupby_df_list = []
        for groupby_col in groupby_cols:
            for feature_col in feature_cols:
                groupby_features_df = groupby_feat_engi(worn_df, groupby_col, feature_col)
                groupby_df_list.append(groupby_features_df)
    
        groupby_concat_df = pd.concat(groupby_df_list, axis=1)
    
        uncommon_groupby_cols = ['day_since_wear', 'timestamp_hour_window']#, 'timestamp_15_min_window']#, 'timestamp_min_window']
    
        uncommon_groupby_df_list = []
        for uncommon_groupby_col in uncommon_groupby_cols:
            for unc_feature_col in feature_cols:
                uncommon_groupby_df = uncommon_groupby_feat_engi(worn_df, uncommon_groupby_col, unc_feature_col)
                uncommon_groupby_df_list.append(uncommon_groupby_df)
    
        features_concat_df = pd.concat([features_df, groupby_concat_df, uncommon_groupby_df], axis=1)
        features_current_cols = features_concat_df.columns.tolist()
        features_concat_df['id'] = iid
        features_concat_df = features_concat_df[['id'] + features_current_cols]
    
        for q in [1,2,3,4]:
            if q in worn_df['quarter'].tolist():
                features_concat_df['data_collected_in_quarter_' + str(q)] = 1
            else:
                features_concat_df['data_collected_in_quarter_' + str(q)] = 0

        features_concat_df['id'] = features_concat_df['id'].astype(str)

        #features_concat_df.to_csv(path + 'series_' + dataset + '.parquet/id=' + iid + '/features.csv', index=False)
        
        ts_list.append(parquet_concat_df)
        ts_features_list.append(features_concat_df)

        gc.collect()
    gc.collect()
    
    return ts_list, ts_features_list

In [ ]:
def read_features_files():
    temp_dfs_list = []
    ts_features_folders_list = os.listdir(path + 'train_ts_features/')
    for temp_id_folder in tqdm(ts_features_folders_list):
        temp_files_list = os.listdir(path + 'train_ts_features/' + temp_id_folder + '/')
        for temp_filename in temp_files_list:
            if '.csv' in temp_filename:
                temp_df = pd.read_csv(path + 'train_ts_features/' + temp_id_folder + '/' + temp_filename, dtype={'id': str})
                temp_dfs_list.append(temp_df)
    gc.collect()
    return temp_dfs_list

In [ ]:
train_ts_features_list = read_features_files()

In [ ]:
test_ts_list, test_ts_features_list = read_parquet(dataset='test')

In [ ]:
train_ts_features_concat_df = pd.concat(train_ts_features_list, ignore_index=True)
test_ts_features_concat_df = pd.concat(test_ts_features_list, ignore_index=True)

In [ ]:
del train_ts_features_list, test_ts_features_list
gc.collect()

In [ ]:
train_merge_df = train_temp_df.merge(train_ts_features_concat_df, how='left', on='id')
test_merge_df = test_temp_df.merge(test_ts_features_concat_df, how='left', on='id')

In [ ]:
del train_ts_features_concat_df, test_ts_features_concat_df
gc.collect()

In [ ]:
cat_cols = []
for i in range(data_dict.shape[0]):
    col = data_dict['Field'][i]
    if col == 'id':
        continue
    if (data_dict['Type'][i] == 'str') | ('categori' in data_dict['Type'][i]):
        cat_cols.append(col)

In [ ]:
ordinal_variables = ['FGC-FGC_CU_Zone', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone',
                     'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_Frame_num',
                     'PreInt_EduHx-computerinternet_hoursday']

ordinal_variables += [col for col in cat_cols if (('PCIAT' in col)&('Season' not in col))]

In [ ]:
cat_cols = list(set(cat_cols) - set(ordinal_variables))
cat_cols = [col for col in cat_cols if 'PCIAT' not in col]

In [ ]:
merge_concat_df = pd.concat([train_merge_df, test_merge_df], ignore_index=True)

In [ ]:
del train_merge_df, test_merge_df
gc.collect()

In [ ]:
def remove_pciat_cols(df):
    cols_to_drop = []
    for col in df.columns:
        if 'PCIAT' in col:
            cols_to_drop.append(col)

    df.drop(cols_to_drop, axis=1, inplace=True)
    
    return df

In [ ]:
merge_concat_df = remove_pciat_cols(merge_concat_df)

In [ ]:
def one_hot_cat_cols(df):
    for col in tqdm(cat_cols):
        df[col] = df[col].fillna('Missing').apply(str)
        unique_list = sorted(df[col].unique().tolist())

        for v in unique_list:
            new_col_name = col + '_' + v
            df[new_col_name] = (df[col] == v).astype(int)
    
    df = df.drop(cat_cols, axis=1)

    return df

In [ ]:
merge_concat_df = one_hot_cat_cols(merge_concat_df)

In [ ]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
def feature_engineering(df):
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    
    return df

In [ ]:
merge_concat_df = feature_engineering(merge_concat_df)

In [ ]:
no_id_cols = [col for col in merge_concat_df.columns if col != 'id']

In [ ]:
if np.any(np.isinf(merge_concat_df[no_id_cols])):
    print('inf in data')
    merge_concat_df[no_id_cols] = merge_concat_df[no_id_cols].replace([np.inf, -np.inf], np.nan)

In [ ]:
imputer = KNNImputer(n_neighbors=5)
merge_concat_df[no_id_cols] = imputer.fit_transform(merge_concat_df[no_id_cols])

In [ ]:
del imputer
gc.collect()

In [ ]:
train_df = merge_concat_df.head(train.shape[0]).reset_index(drop=True)
test_df = merge_concat_df.tail(test.shape[0]).reset_index(drop=True)

In [ ]:
train_df['sii'] = train['sii']

In [ ]:
del train, test
gc.collect()

In [ ]:
train_nonan_df = train_df[~train_df['sii'].isna()].reset_index(drop=True)

In [ ]:
del train_df
gc.collect()

In [ ]:
#selected_features_df = pd.read_csv(path + 'selected_features.csv')

In [ ]:
#train_cols = selected_features_df['selected_features'].tolist()

In [ ]:
#del selected_features_df
#gc.collect()

In [ ]:
#test_cols = train_cols.copy()
#test_cols.remove('sii')

In [ ]:
#train_final_df = train_nonan_df[train_cols].copy()
#test_final_df = test_df[test_cols].copy()

In [ ]:
train_final_df = train_nonan_df.copy()
test_final_df = test_df.copy()

In [ ]:
del train_nonan_df, test_df
gc.collect()

In [ ]:
#selected_cols_df = pd.read_csv(path + 'selected_cols.csv')
#selected_cols_df

In [ ]:
#selected_cols = selected_cols_df['col'].tolist()

In [ ]:
#train_final_df = train_final_df[selected_cols + ['sii']]
#print(train_final_df.shape)
#train_final_df.head()

In [ ]:
#test_final_df = test_final_df[selected_cols]
#print(test_final_df.shape)
#test_final_df.head()

In [ ]:
corr_df = train_final_df.drop(['id', 'sii'], axis=1).corr()

In [ ]:
# Find features with high correlation
high_corr = np.where(corr_df > 0.9)
high_corr_pairs = [
    (corr_df.index[x], corr_df.columns[y])
    for x, y in zip(*high_corr)
    if x != y and x < y
]

In [ ]:
second_v_list = []
for v in high_corr_pairs:
    second_v = v[1]
    second_v_list.append(second_v)

In [ ]:
redundant_features = list(set(second_v_list))

In [ ]:
train_final_df = train_final_df.drop(redundant_features, axis=1)
print(train_final_df.shape)
train_final_df.head()

In [ ]:
test_final_df = test_final_df.drop(redundant_features, axis=1)
print(test_final_df.shape)
test_final_df.head()

# Parameter tuning, training, prediction & submission

In [8]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

In [9]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
# Edited by Max (Jongyun Han)
def catboost_param_tune(train_data, test_data, optuna_n_trials):
    X = train_data.drop(['id', 'sii'], axis=1)
    y = train_data['sii']

    test_data_copy = test_data.copy()
    test_data = test_data.drop('id', axis=1)

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
     
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    models_list = []

    base_params = {
        'iterations': 1500,
        'random_seed': SEED,
        'verbose': 0,
        'task_type': 'GPU',
        'use_best_model': True,
        'boosting_type': 'Plain'
    }

    def objective(trial):

        oof_non_rounded = np.zeros(len(y), dtype=float)
        
        params_to_tune = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.7),
            'depth': trial.suggest_int('depth', 6, 8),
            'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 2000),
            'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 10, 70),
            #'border_count': trial.suggest_int('border_count', 64, 512),
            'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 2),
            'random_strength': trial.suggest_float('random_strength', 0, 20)
        }
    
        #for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
            X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
    
            model = CatBoostRegressor(
                **base_params,
                **params_to_tune
            )
            
            model.fit(X_train, y_train, eval_set=(X_val, y_val))#, plot=True)
    
            y_train_pred = model.predict(X_train)
            y_val_pred = model.predict(X_val)
    
            oof_non_rounded[test_idx] = y_val_pred
            y_val_pred_rounded = y_val_pred.round(0).astype(int)
            oof_rounded[test_idx] = y_val_pred_rounded

        # https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/discussion/551533
        y_preds_df = pd.DataFrame({'y':y, 'preds':oof_non_rounded})
        oof_initial_thresholds = y_preds_df.groupby('y')['preds'].mean().iloc[1:].values.tolist()
    
        KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=oof_initial_thresholds, args=(y, oof_non_rounded), 
                                  method='Nelder-Mead')
        assert KappaOPtimizer.success, "Optimization did not converge."
        
        oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
        tKappa = quadratic_weighted_kappa(y, oof_tuned)

        del y_preds_df, oof_initial_thresholds
        gc.collect()
    
        return tKappa

    with tqdm(total=optuna_n_trials, desc="Optimizing", unit="trial") as pbar:
        
        # Define a callback function to update the progress bar
        def progress_bar_callback(study, trial):
            pbar.update(1)
    
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=optuna_n_trials, callbacks=[progress_bar_callback])

    best_params = study.best_params

    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    models_list = []
    shap_importance_list = []

    #for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
    for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = CatBoostRegressor(
            **base_params,
            **best_params
        )
        
        model.fit(X_train, y_train, eval_set=(X_val, y_val), plot=True)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_y_preds_df = pd.DataFrame({'y':y_train, 'preds':y_train_pred})
        train_oof_initial_thresholds = train_y_preds_df.groupby('y')['preds'].mean().iloc[1:].values.tolist()

        train_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=train_oof_initial_thresholds, args=(y_train, y_train_pred), 
                                  method='Nelder-Mead')
        assert train_KappaOPtimizer.success, "Train Optimization did not converge."

        train_pred_tuned = threshold_Rounder(y_train_pred, train_KappaOPtimizer.x)
        train_Kappa = quadratic_weighted_kappa(y_train, train_pred_tuned)

        print(f"----> || Train Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_Kappa:.3f}{Style.RESET_ALL}")

        val_y_preds_df = pd.DataFrame({'y':y_val, 'preds':y_val_pred})
        val_oof_initial_thresholds = val_y_preds_df.groupby('y')['preds'].mean().iloc[1:].values.tolist()

        val_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=val_oof_initial_thresholds, args=(y_val, y_val_pred), 
                                  method='Nelder-Mead')
        assert val_KappaOPtimizer.success, "Val Optimization did not converge."

        val_pred_tuned = threshold_Rounder(y_val_pred, val_KappaOPtimizer.x)
        val_Kappa = quadratic_weighted_kappa(y_val, val_pred_tuned)

        print(f"----> || Val Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_Kappa:.3f}{Style.RESET_ALL}")

        train_optimized_val_pred_tuned = threshold_Rounder(y_val_pred, train_KappaOPtimizer.x)
        train_optimized_val_Kappa = quadratic_weighted_kappa(y_val, train_optimized_val_pred_tuned)

        print(f"----> || Train Optimized Val QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_optimized_val_Kappa:.3f}{Style.RESET_ALL}")

        val_optimized_train_pred_tuned = threshold_Rounder(y_train_pred, val_KappaOPtimizer.x)
        val_optimized_train_Kappa = quadratic_weighted_kappa(y_train, val_optimized_train_pred_tuned)

        print(f"----> || Val Optimized Train QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_optimized_train_Kappa:.3f}{Style.RESET_ALL}")

        rounded_train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        rounded_val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)        

        train_S.append(train_Kappa)
        test_S.append(val_Kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Rounded Train QWK: {rounded_train_kappa:.4f}, Rounded Validation QWK: {rounded_val_kappa:.4f}")
        #clear_output(wait=True)

        models_list.append(model)

        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X=X_val, y=y_val)
        shap_importance = np.abs(shap_values).mean(axis=0)

        shap_importance_list.append(shap_importance)

        

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    y_preds_df = pd.DataFrame({'y':y, 'preds':oof_non_rounded})
    oof_initial_thresholds = y_preds_df.groupby('y')['preds'].mean().iloc[1:].values.tolist()

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=oof_initial_thresholds, args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': test_data_copy['id'],
        'sii': tpTuned
    })

    return models_list, submission, oof_non_rounded, tpm, study, np.mean(shap_importance_list, axis=0)

In [10]:
cat_models_list, cat_submission_df, cat_raw_train_preds, cat_raw_test_preds, cat_optuna_study, cat_shap_importance = catboost_param_tune(
    train_final_df, test_final_df, 300
)

Optimizing: 100%|███████████████████████████████████████████████████████████████| 300/300 [1:49:16<00:00, 21.86s/trial]


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

----> || Train Optimized QWK SCORE ::  0.848
----> || Val Optimized QWK SCORE ::  0.547
----> || Train Optimized Val QWK SCORE ::  0.500
----> || Val Optimized Train QWK SCORE ::  0.806
Fold 1 - Rounded Train QWK: 0.7618, Rounded Validation QWK: 0.3857


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

----> || Train Optimized QWK SCORE ::  0.869
----> || Val Optimized QWK SCORE ::  0.556
----> || Train Optimized Val QWK SCORE ::  0.495
----> || Val Optimized Train QWK SCORE ::  0.856
Fold 2 - Rounded Train QWK: 0.7969, Rounded Validation QWK: 0.4632


...

----> || Train Optimized QWK SCORE ::  0.866
----> || Val Optimized QWK SCORE ::  0.540
----> || Train Optimized Val QWK SCORE ::  0.481
----> || Val Optimized Train QWK SCORE ::  0.844
Fold 3 - Rounded Train QWK: 0.7840, Rounded Validation QWK: 0.4231


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

----> || Train Optimized QWK SCORE ::  0.826
----> || Val Optimized QWK SCORE ::  0.488
----> || Train Optimized Val QWK SCORE ::  0.466
----> || Val Optimized Train QWK SCORE ::  0.792
Fold 4 - Rounded Train QWK: 0.7208, Rounded Validation QWK: 0.3347


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

----> || Train Optimized QWK SCORE ::  0.883
----> || Val Optimized QWK SCORE ::  0.432
----> || Train Optimized Val QWK SCORE ::  0.443
----> || Val Optimized Train QWK SCORE ::  0.820
Fold 5 - Rounded Train QWK: 0.8259, Rounded Validation QWK: 0.4263
Mean Train QWK --> 0.8584
Mean Validation QWK ---> 0.5126
----> || Optimized QWK SCORE ::  0.511


In [ ]:
for param in cat_optuna_study.best_params.keys():
    fig = plot_slice(cat_optuna_study, params=[param])
    fig.show()

In [ ]:
plot_param_importances(cat_optuna_study)

In [ ]:
cat_optuna_study.best_params

In [ ]:
cols = train_final_df.drop(['id', 'sii'], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, cat_shap_importance)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
unimportant_df = imp_df[imp_df['Importance'] <= imp_df['Importance'].quantile(0.5)]
unimportant_cols = unimportant_df['Feature'].tolist()

In [ ]:
train_selected_df = train_final_df.drop(unimportant_cols, axis=1)
print(train_selected_df.shape)
train_selected_df.head()

In [ ]:
test_selected_df = test_final_df.drop(unimportant_cols, axis=1)
print(test_selected_df.shape)
test_selected_df.head()

In [ ]:
cat_models_list2, cat_submission_df, cat_raw_train_preds, cat_raw_test_preds, cat_optuna_study, cat_shap_importance2 = catboost_param_tune(
    train_selected_df, test_selected_df, 300
)

In [ ]:
for param in cat_optuna_study.best_params.keys():
    fig = plot_slice(cat_optuna_study, params=[param])
    fig.show()

In [ ]:
plot_param_importances(cat_optuna_study)

In [ ]:
cat_optuna_study.best_params

In [ ]:
cols = train_selected_df.drop(['id', 'sii'], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, cat_shap_importance2)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
unimportant_df = imp_df[imp_df['Importance'] <= imp_df['Importance'].quantile(0.1)]
unimportant_cols = unimportant_df['Feature'].tolist()

In [ ]:
train_selected_df3 = train_selected_df.drop(unimportant_cols, axis=1)
print(train_selected_df3.shape)
train_selected_df3.head()

In [ ]:
test_selected_df3 = test_selected_df.drop(unimportant_cols, axis=1)
print(test_selected_df3.shape)
test_selected_df3.head()

In [ ]:
cat_models_list3, cat_submission_df, cat_raw_train_preds, cat_raw_test_preds, cat_optuna_study, cat_shap_importance3 = catboost_param_tune(
    train_selected_df3, test_selected_df3, 300
)

In [ ]:
for param in cat_optuna_study.best_params.keys():
    fig = plot_slice(cat_optuna_study, params=[param])
    fig.show()

In [ ]:
plot_param_importances(cat_optuna_study)

In [ ]:
cat_optuna_study.best_params

In [ ]:
cols = train_selected_df3.drop(['id', 'sii'], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, cat_shap_importance3)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
unimportant_df = imp_df[imp_df['Importance'] <= imp_df['Importance'].quantile(0.1)]
unimportant_cols = unimportant_df['Feature'].tolist()

In [ ]:
train_selected_df4 = train_selected_df3.drop(unimportant_cols, axis=1)
print(train_selected_df4.shape)
train_selected_df4.head()

In [ ]:
test_selected_df4 = test_selected_df3.drop(unimportant_cols, axis=1)
print(test_selected_df4.shape)
test_selected_df4.head()

In [ ]:
cat_models_list4, cat_submission_df, cat_raw_train_preds, cat_raw_test_preds, cat_optuna_study, cat_shap_importance4 = catboost_param_tune(
    train_selected_df4, test_selected_df4, 300
)

In [ ]:
for param in cat_optuna_study.best_params.keys():
    fig = plot_slice(cat_optuna_study, params=[param])
    fig.show()

In [ ]:
plot_param_importances(cat_optuna_study)

In [ ]:
cat_optuna_study.best_params

In [ ]:
cols = train_selected_df4.drop(['id', 'sii'], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, cat_shap_importance4)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
unimportant_df = imp_df[imp_df['Importance'] <= imp_df['Importance'].quantile(0.1)]
unimportant_cols = unimportant_df['Feature'].tolist()

In [ ]:
train_selected_df5 = train_selected_df4.drop(unimportant_cols, axis=1)
print(train_selected_df5.shape)
train_selected_df5.head()

In [ ]:
test_selected_df5 = test_selected_df4.drop(unimportant_cols, axis=1)
print(test_selected_df5.shape)
test_selected_df5.head()

In [ ]:
cat_models_list5, cat_submission_df, cat_raw_train_preds, cat_raw_test_preds, cat_optuna_study, cat_shap_importance5 = catboost_param_tune(
    train_selected_df5, test_selected_df5, 250
)

In [ ]:
for param in cat_optuna_study.best_params.keys():
    fig = plot_slice(cat_optuna_study, params=[param])
    fig.show()

In [ ]:
plot_param_importances(cat_optuna_study)

In [ ]:
cat_optuna_study.best_params

In [ ]:
cols = train_selected_df5.drop(['id', 'sii'], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, cat_shap_importance5)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
unimportant_df = imp_df[imp_df['Importance'] <= imp_df['Importance'].quantile(0.1)]
unimportant_cols = unimportant_df['Feature'].tolist()

In [ ]:
train_selected_df6 = train_selected_df5.drop(unimportant_cols, axis=1)
print(train_selected_df6.shape)
train_selected_df6.head()

In [ ]:
test_selected_df6 = test_selected_df5.drop(unimportant_cols, axis=1)
print(test_selected_df6.shape)
test_selected_df6.head()

In [ ]:
cat_models_list6, cat_submission_df, cat_raw_train_preds, cat_raw_test_preds, cat_optuna_study, cat_shap_importance6 = catboost_param_tune(
    train_selected_df6, test_selected_df6, 300
)

In [ ]:
for param in cat_optuna_study.best_params.keys():
    fig = plot_slice(cat_optuna_study, params=[param])
    fig.show()

In [ ]:
plot_param_importances(cat_optuna_study)

In [ ]:
cat_optuna_study.best_params

In [ ]:
cols = train_selected_df6.drop(['id', 'sii'], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, cat_shap_importance6)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
unimportant_df = imp_df[imp_df['Importance'] <= imp_df['Importance'].quantile(0.1)]
unimportant_cols = unimportant_df['Feature'].tolist()

In [ ]:
train_selected_df7 = train_selected_df6.drop(unimportant_cols, axis=1)
print(train_selected_df7.shape)
train_selected_df7.head()

In [ ]:
test_selected_df7 = test_selected_df6.drop(unimportant_cols, axis=1)
print(test_selected_df7.shape)
test_selected_df7.head()

In [ ]:
cat_models_list7, cat_submission_df, cat_raw_train_preds, cat_raw_test_preds, cat_optuna_study, cat_shap_importance7 = catboost_param_tune(
    train_selected_df7, test_selected_df7, 300
)

In [ ]:
for param in cat_optuna_study.best_params.keys():
    fig = plot_slice(cat_optuna_study, params=[param])
    fig.show()

In [ ]:
plot_param_importances(cat_optuna_study)

In [ ]:
cat_optuna_study.best_params

In [ ]:
cols = train_selected_df7.drop(['id', 'sii'], axis=1).columns
imp_df = pd.DataFrame(sorted(zip(cols, cat_shap_importance7)), columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)
unimportant_df = imp_df[imp_df['Importance'] <= imp_df['Importance'].quantile(0.1)]
unimportant_cols = unimportant_df['Feature'].tolist()

In [ ]:
train_selected_df8 = train_selected_df7.drop(unimportant_cols, axis=1)
print(train_selected_df8.shape)
train_selected_df8.head()

In [ ]:
test_selected_df8 = test_selected_df7.drop(unimportant_cols, axis=1)
print(test_selected_df8.shape)
test_selected_df8.head()

In [ ]:
train_selected_df7.to_csv(path + 'cat_train_selected_7.csv', index=False)

In [ ]:
test_selected_df7.to_csv(path + 'cat_test_selected_7.csv', index=False)

In [ ]:
train_selected_df8.to_csv(path + 'cat_train_selected_8.csv', index=False)

In [ ]:
test_selected_df8.to_csv(path + 'cat_test_selected_8.csv', index=False)

In [ ]:
cat_models_list8, cat_submission_df, cat_raw_train_preds, cat_raw_test_preds, cat_optuna_study, cat_shap_importance8 = catboost_param_tune(
    train_selected_df8, test_selected_df8, 300
)

In [ ]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
# Edited by Max (Jongyun Han)

def lgb_feature_selection(train_data, test_data, optuna_n_trials):
    X = train_data.drop(['id', 'sii'], axis=1)
    y = train_data['sii']

    test_data_copy = test_data.copy()
    test_data = test_data.drop('id', axis=1)

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    models_list = []

    base_params = {
        'verbosity': -1,
        'seed': SEED,
        'num_iterations': 10000,
        'feature_fraction': 0.8,
        'device': 'gpu'
    }

    def objective(trial):

        params_to_tune = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'early_stopping_round': trial.suggest_int('early_stopping_round', 10, 70),
            'max_depth': trial.suggest_int('max_depth', 6, 50),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 300),
            'num_leaves': trial.suggest_int('num_leaves', 50, 10000),
            'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 0.3),
            'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
            'lambda_l2': trial.suggest_float('lambda_l2', 50, 2000)
        }

        #for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
            X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
            
            lgb_callbacks = [
                lgb.log_evaluation(period=20)
            ]
    
            model = LGBMRegressor(
                **base_params,
                **params_to_tune
            )
            
            model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)])#, callbacks=lgb_callbacks)
    
            #plt.figure()
            #lgb.plot_metric(model)
            #plt.show()
            
            y_train_pred = model.predict(X_train)
            y_val_pred = model.predict(X_val)
    
            oof_non_rounded[test_idx] = y_val_pred
            y_val_pred_rounded = y_val_pred.round(0).astype(int)
            oof_rounded[test_idx] = y_val_pred_rounded
    
        #print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
        #print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

        # https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/discussion/551533
        y_preds_df = pd.DataFrame({'y':y, 'preds':oof_non_rounded})
        oof_initial_thresholds = y_preds_df.groupby('y')['preds'].mean().iloc[1:].values.tolist()
    
        KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=oof_initial_thresholds, args=(y, oof_non_rounded), 
                                  method='Nelder-Mead')
        assert KappaOPtimizer.success, "Optimization did not converge."

        del y_preds_df, oof_initial_thresholds
        gc.collect()
        
        oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
        tKappa = quadratic_weighted_kappa(y, oof_tuned)
    
        #print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
        return tKappa

    with tqdm(total=optuna_n_trials, desc="Optimizing", unit="trial") as pbar:
        
        # Define a callback function to update the progress bar
        def progress_bar_callback(study, trial):
            pbar.update(1)
    
        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=optuna_n_trials, callbacks=[progress_bar_callback])

    best_params = study.best_params

    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    models_list = []

    for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
        
        lgb_callbacks = [
            lgb.log_evaluation(period=20)
        ]

        model = LGBMRegressor(
            **base_params,
            **best_params
        )
        
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)])#, callbacks=lgb_callbacks)

        plt.figure()
        lgb.plot_metric(model)
        plt.show()
        
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_y_preds_df = pd.DataFrame({'y':y_train, 'preds':y_train_pred})
        train_oof_initial_thresholds = train_y_preds_df.groupby('y')['preds'].mean().iloc[1:].values.tolist()

        train_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=train_oof_initial_thresholds, args=(y_train, y_train_pred), 
                                  method='Nelder-Mead')
        assert train_KappaOPtimizer.success, "Train Optimization did not converge."

        train_pred_tuned = threshold_Rounder(y_train_pred, train_KappaOPtimizer.x)
        train_Kappa = quadratic_weighted_kappa(y_train, train_pred_tuned)

        print(f"----> || Train Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_Kappa:.3f}{Style.RESET_ALL}")

        val_y_preds_df = pd.DataFrame({'y':y_val, 'preds':y_val_pred})
        val_oof_initial_thresholds = val_y_preds_df.groupby('y')['preds'].mean().iloc[1:].values.tolist()

        val_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=val_oof_initial_thresholds, args=(y_val, y_val_pred), 
                                  method='Nelder-Mead')
        assert val_KappaOPtimizer.success, "Val Optimization did not converge."

        val_pred_tuned = threshold_Rounder(y_val_pred, val_KappaOPtimizer.x)
        val_Kappa = quadratic_weighted_kappa(y_val, val_pred_tuned)

        print(f"----> || Val Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_Kappa:.3f}{Style.RESET_ALL}")

        train_optimized_val_pred_tuned = threshold_Rounder(y_val_pred, train_KappaOPtimizer.x)
        train_optimized_val_Kappa = quadratic_weighted_kappa(y_val, train_optimized_val_pred_tuned)

        print(f"----> || Train Optimized Val QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_optimized_val_Kappa:.3f}{Style.RESET_ALL}")

        val_optimized_train_pred_tuned = threshold_Rounder(y_train_pred, val_KappaOPtimizer.x)
        val_optimized_train_Kappa = quadratic_weighted_kappa(y_train, val_optimized_train_pred_tuned)

        print(f"----> || Val Optimized Train QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_optimized_train_Kappa:.3f}{Style.RESET_ALL}")

        rounded_train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        rounded_val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)        

        train_S.append(train_Kappa)
        test_S.append(val_Kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Rounded Train QWK: {rounded_train_kappa:.4f}, Rounded Validation QWK: {rounded_val_kappa:.4f}")
        #clear_output(wait=True)

        models_list.append(model)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    y_preds_df = pd.DataFrame({'y':y, 'preds':oof_non_rounded})
    oof_initial_thresholds = y_preds_df.groupby('y')['preds'].mean().iloc[1:].values.tolist()

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=oof_initial_thresholds, args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': test_data_copy['id'],
        'sii': tpTuned
    })

    return models_list, submission, oof_non_rounded, tpm, study

In [ ]:
lgb_models_list, lgb_submission_df, lgb_raw_train_preds, lgb_raw_test_preds, lgb_optuna_study = lgb_feature_selection(train_final_df, test_final_df, 300)

In [ ]:
lgb_params_dict = {
    'verbosity': [-1],
    'seed': [SEED],
    'num_iterations': [10000],
    'feature_fraction': [0.8],
    'device': ['gpu'],
    'learning_rate': [0.25260664212527056],
    'early_stopping_round': [60],
    'max_depth': [7],
    'min_data_in_leaf': [183],
    'num_leaves': [6388],
    'min_gain_to_split': [0.05311446129934012],
    'lambda_l1': [0.45136290968123227],
    'lambda_l2': [804.4678501621871]
}

In [ ]:
lgb_params_df = pd.DataFrame(lgb_params_dict)
print(lgb_params_df.shape)
lgb_params_df

In [ ]:
lgb_params_df.to_csv(path + 'lgb_params.csv', index=False)

In [ ]:
selected_cols = test_selected_df8.columns.tolist()
selected_cols_df = pd.DataFrame({'col':selected_cols})
selected_cols_df.to_csv(path + 'selected_cols.csv', index=False)

In [ ]:
lgb_models_list8, lgb_submission_df, lgb_raw_train_preds, lgb_raw_test_preds, lgb_optuna_study = lgb_feature_selection(
    train_selected_df8, test_selected_df8, 3
)

In [ ]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
# Edited by Max (Jongyun Han)
def lgb_train(params, train_data, test_data):
    X = train_data.drop(['id', 'sii'], axis=1)
    y = train_data['sii']

    test_data_copy = test_data.copy()
    test_data = test_data.drop('id', axis=1)

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    models_list = []

    #for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
    for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
        
        lgb_callbacks = [
            lgb.log_evaluation(period=20)
        ]

        model = LGBMRegressor(
            **params
        )
        
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)])#, callbacks=lgb_callbacks)

        plt.figure()
        lgb.plot_metric(model)
        plt.show()
        
        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=[0.5, 1.5, 2.5], args=(y_train, y_train_pred), 
                                  method='Nelder-Mead')
        assert train_KappaOPtimizer.success, "Train Optimization did not converge."

        train_pred_tuned = threshold_Rounder(y_train_pred, train_KappaOPtimizer.x)
        train_Kappa = quadratic_weighted_kappa(y_train, train_pred_tuned)

        print(f"----> || Train Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_Kappa:.3f}{Style.RESET_ALL}")

        val_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=[0.5, 1.5, 2.5], args=(y_val, y_val_pred), 
                                  method='Nelder-Mead')
        assert val_KappaOPtimizer.success, "Val Optimization did not converge."

        val_pred_tuned = threshold_Rounder(y_val_pred, val_KappaOPtimizer.x)
        val_Kappa = quadratic_weighted_kappa(y_val, val_pred_tuned)

        print(f"----> || Val Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_Kappa:.3f}{Style.RESET_ALL}")

        train_optimized_val_pred_tuned = threshold_Rounder(y_val_pred, train_KappaOPtimizer.x)
        train_optimized_val_Kappa = quadratic_weighted_kappa(y_val, train_optimized_val_pred_tuned)

        print(f"----> || Train Optimized Val QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_optimized_val_Kappa:.3f}{Style.RESET_ALL}")

        val_optimized_train_pred_tuned = threshold_Rounder(y_train_pred, val_KappaOPtimizer.x)
        val_optimized_train_Kappa = quadratic_weighted_kappa(y_train, val_optimized_train_pred_tuned)

        print(f"----> || Val Optimized Train QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_optimized_train_Kappa:.3f}{Style.RESET_ALL}")

        rounded_train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        rounded_val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)        

        train_S.append(train_Kappa)
        test_S.append(val_Kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Rounded Train QWK: {rounded_train_kappa:.4f}, Rounded Validation QWK: {rounded_val_kappa:.4f}")
        #clear_output(wait=True)

        models_list.append(model)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': test_data_copy['id'],
        'sii': tpTuned
    })

    return models_list, submission, oof_non_rounded, tpm

In [ ]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
# Edited by Max (Jongyun Han)
def catboost_feature_selection(train_data, test_data, optuna_n_trials=100):
    X = train_data.drop(['id', 'sii'], axis=1)
    y = train_data['sii']

    test_data_copy = test_data.copy()
    test_data = test_data.drop('id', axis=1)

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    models_list = []

    base_params = {
        'iterations': 3000,
        'random_seed': SEED,
        'verbose': 0,
        'task_type': 'GPU',
        'use_best_model': True,
        'boosting_type': 'Plain'
    }

    def objective(trial):
        
        params_to_tune = {
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.03),
            'depth': trial.suggest_int('depth', 6, 9),
            'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 10, 70),
            'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 20, 40)
        }
    
        #for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
            X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]
    
            model = CatBoostRegressor(
                **base_params,
                **params_to_tune
            )
            
            model.fit(X_train, y_train, eval_set=(X_val, y_val))#, plot=True)
    
            y_train_pred = model.predict(X_train)
            y_val_pred = model.predict(X_val)
    
            oof_non_rounded[test_idx] = y_val_pred
            y_val_pred_rounded = y_val_pred.round(0).astype(int)
            oof_rounded[test_idx] = y_val_pred_rounded
    
            train_KappaOPtimizer = minimize(evaluate_predictions,
                                      x0=[0.5, 1.5, 2.5], args=(y_train, y_train_pred), 
                                      method='Nelder-Mead')
            assert train_KappaOPtimizer.success, "Train Optimization did not converge."
    
            train_pred_tuned = threshold_Rounder(y_train_pred, train_KappaOPtimizer.x)
            train_Kappa = quadratic_weighted_kappa(y_train, train_pred_tuned)
    
            #print(f"----> || Train Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_Kappa:.3f}{Style.RESET_ALL}")
    
            val_KappaOPtimizer = minimize(evaluate_predictions,
                                      x0=[0.5, 1.5, 2.5], args=(y_val, y_val_pred), 
                                      method='Nelder-Mead')
            assert val_KappaOPtimizer.success, "Val Optimization did not converge."
    
            val_pred_tuned = threshold_Rounder(y_val_pred, val_KappaOPtimizer.x)
            val_Kappa = quadratic_weighted_kappa(y_val, val_pred_tuned)
    
            #print(f"----> || Val Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_Kappa:.3f}{Style.RESET_ALL}")
    
            train_optimized_val_pred_tuned = threshold_Rounder(y_val_pred, train_KappaOPtimizer.x)
            train_optimized_val_Kappa = quadratic_weighted_kappa(y_val, train_optimized_val_pred_tuned)
    
            #print(f"----> || Train Optimized Val QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_optimized_val_Kappa:.3f}{Style.RESET_ALL}")
    
            val_optimized_train_pred_tuned = threshold_Rounder(y_train_pred, val_KappaOPtimizer.x)
            val_optimized_train_Kappa = quadratic_weighted_kappa(y_train, val_optimized_train_pred_tuned)
    
            #print(f"----> || Val Optimized Train QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_optimized_train_Kappa:.3f}{Style.RESET_ALL}")
    
            rounded_train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
            rounded_val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)        
    
            train_S.append(train_Kappa)
            test_S.append(val_Kappa)
            
            test_preds[:, fold] = model.predict(test_data)
            
            #print(f"Fold {fold+1} - Rounded Train QWK: {rounded_train_kappa:.4f}, Rounded Validation QWK: {rounded_val_kappa:.4f}")
            #clear_output(wait=True)
    
            models_list.append(model)
    
        #print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
        #print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")
    
        KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                                  method='Nelder-Mead')
        assert KappaOPtimizer.success, "Optimization did not converge."
        
        oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
        tKappa = quadratic_weighted_kappa(y, oof_tuned)
    
        print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
        tpm = test_preds.mean(axis=1)
        tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
        
        submission = pd.DataFrame({
            'id': test_data_copy['id'],
            'sii': tpTuned
        })
    
        return tKappa

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=optuna_n_trials)  # You can increase n_trials for better results

    return study

In [ ]:
catboost_optuna_study = catboost_feature_selection(train_final_df, test_final_df, 5)

In [ ]:
plot_optimization_history(catboost_optuna_study)

In [ ]:
plot_optimization_history(catboost_optuna_study)

In [ ]:
plot_parallel_coordinate(catboost_optuna_study)

In [ ]:
plot_slice(catboost_optuna_study)

In [ ]:
plot_param_importances(catboost_optuna_study)

In [ ]:
plot_rank(catboost_optuna_study)

In [ ]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
# Edited by Max (Jongyun Han)
def catboost_train(train_data, test_data):
    X = train_data.drop(['id', 'sii'], axis=1)
    y = train_data['sii']

    test_data_copy = test_data.copy()
    test_data = test_data.drop('id', axis=1)

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    models_list = []

    catboost_params = {
        'learning_rate': 0.02,
        'depth': 9,
        'iterations': 2000,
        'random_seed': SEED,
        'verbose': 0,
        'l2_leaf_reg': 15,
        'task_type': 'GPU',
        'use_best_model': True,
        'early_stopping_rounds': 30,
        'boosting_type': 'Ordered'
    }

    #for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
    for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = CatBoostRegressor(
            **catboost_params
        )
        
        model.fit(X_train, y_train, eval_set=(X_val, y_val), plot=True)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=[0.5, 1.5, 2.5], args=(y_train, y_train_pred), 
                                  method='Nelder-Mead')
        assert train_KappaOPtimizer.success, "Train Optimization did not converge."

        train_pred_tuned = threshold_Rounder(y_train_pred, train_KappaOPtimizer.x)
        train_Kappa = quadratic_weighted_kappa(y_train, train_pred_tuned)

        print(f"----> || Train Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_Kappa:.3f}{Style.RESET_ALL}")

        val_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=[0.5, 1.5, 2.5], args=(y_val, y_val_pred), 
                                  method='Nelder-Mead')
        assert val_KappaOPtimizer.success, "Val Optimization did not converge."

        val_pred_tuned = threshold_Rounder(y_val_pred, val_KappaOPtimizer.x)
        val_Kappa = quadratic_weighted_kappa(y_val, val_pred_tuned)

        print(f"----> || Val Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_Kappa:.3f}{Style.RESET_ALL}")

        train_optimized_val_pred_tuned = threshold_Rounder(y_val_pred, train_KappaOPtimizer.x)
        train_optimized_val_Kappa = quadratic_weighted_kappa(y_val, train_optimized_val_pred_tuned)

        print(f"----> || Train Optimized Val QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_optimized_val_Kappa:.3f}{Style.RESET_ALL}")

        val_optimized_train_pred_tuned = threshold_Rounder(y_train_pred, val_KappaOPtimizer.x)
        val_optimized_train_Kappa = quadratic_weighted_kappa(y_train, val_optimized_train_pred_tuned)

        print(f"----> || Val Optimized Train QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_optimized_train_Kappa:.3f}{Style.RESET_ALL}")

        rounded_train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        rounded_val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)        

        train_S.append(train_Kappa)
        test_S.append(val_Kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Rounded Train QWK: {rounded_train_kappa:.4f}, Rounded Validation QWK: {rounded_val_kappa:.4f}")
        #clear_output(wait=True)

        models_list.append(model)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': test_data_copy['id'],
        'sii': tpTuned
    })

    return models_list, submission, oof_non_rounded, tpm

In [ ]:
cat_models_list, cat_submission_df, cat_raw_train_preds, cat_raw_test_preds = catboost_train(train_final_df, test_final_df)

In [ ]:
#submission_df.to_csv('submission.csv', index=False)